In [1]:
import anndata
import glob
import numpy as np
import pandas as pd
import os
import pathlib
import protoplast as pt
import ray
import torch

from anndata.experimental import AnnCollection
from protoplast.scrna.anndata.trainer import RayTrainRunner
from protoplast.scrna.anndata.torch_dataloader import DistributedAnnDataset
from protoplast.scrna.anndata.torch_dataloader import cell_line_metadata_cb, DistributedCellLineAnnDataset

from ray.train import Checkpoint
from ray.train.lightning import RayDDPStrategy

✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!


/mnt/hdd2/nam/miniconda3/envs/test/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!


In [2]:
DS_PATHS = ["/mnt/hdd2/tan/tahoe100m/plate1_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad",
           "/mnt/hdd2/tan/tahoe100m/plate2_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad",
           "/mnt/hdd2/tan/tahoe100m/plate3_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad"]
adatas = [anndata.io.read_h5ad(p, backed = "r") for p in DS_PATHS]

In [3]:
# Create a view of all dataset
collection = AnnCollection(adatas, join_vars = "inner")

# Record the cell-lines (output classes) in both datasets
cell_lines = collection.obs.cell_line.unique().tolist()
cell_lines_count = collection.obs.cell_line.nunique()

In [32]:
import lightning.pytorch as pl
import torch
from torch import nn

class LinearClassifier(pl.LightningModule):
    """
    Example model for implementing the cell line linear classifier
    you can write your own model by extending BaseAnnDataLightningModule
    it is highly recommend to extend from this class if you are using
    the DistributedAnnDataset as your loader
    """

    def __init__(self, num_genes, num_classes):
        super().__init__()
        
        hidden_size = 2

        self.model = nn.Linear(num_genes, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)  # Output layer

        self.loss_fn = nn.CrossEntropyLoss()

    # def forward(self, x):
    #     x = self.model(x)
    #     x = self.fc2(x)
    #     return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch

        fc1 = self.model(x)
        logits = self.fc2(fc1)

        loss = self.loss_fn(logits, y)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch

        fc1 = self.model(x)
        logits = self.fc2(fc1)
        
        preds = torch.argmax(logits, dim=1)
        correct = (preds == y).sum().item()
        total = y.size(0)
        acc = correct / total
        self.log("val_acc", acc)
        return acc

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [33]:
thread_per_worker = 12
test_size = 0.0 # We don't have the test step in the model, so we can set this to 0
val_size = 0.2

In [34]:
# Set up training
trainer = RayTrainRunner(
    LinearClassifier,
    DistributedCellLineAnnDataset,
    model_keys = ["num_genes",
                  "num_classes"],
    metadata_cb = cell_line_metadata_cb,
    sparse_key = "X"
)

2025-09-26 10:14:55,736	INFO worker.py:1951 -- Started a local Ray instance.
(pid=1860428) /mnt/hdd2/nam/miniconda3/envs/test/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
(pid=1860428)   import pynvml  # type: ignore[import]


(TrainTrainable pid=1860428) ✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!
(TrainTrainable pid=1860428) ✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!


(RayTrainWorker pid=1860564) /mnt/hdd2/nam/miniconda3/envs/test/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
(RayTrainWorker pid=1860564)   import pynvml  # type: ignore[import]
(RayTrainWorker pid=1860564) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=1860428) Started distributed worker processes: 
(TorchTrainer pid=1860428) - (node_id=346dd367d84f85b73a1d59f0f5f96030e23f4d1aa6efb6684b104ab3, ip=192.168.1.226, pid=1860564) world_rank=0, local_rank=0, node_rank=0


(RayTrainWorker pid=1860564) ✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!
(RayTrainWorker pid=1860564) ✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!
(RayTrainWorker pid=1860564) =========Starting the training on 0 with num threads: 12=========


(RayTrainWorker pid=1860564) 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
(RayTrainWorker pid=1860564) GPU available: True (cuda), used: True
(RayTrainWorker pid=1860564) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=1860564) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=1860564) /mnt/hdd2/nam/miniconda3/envs/test/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.11 /mnt/hdd2/nam/miniconda3/envs/test/lib/python3.1 ...
(RayTrainWorker pid=1860564) You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precisi

...

(RayTrainWorker pid=1860564) LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
(RayTrainWorker pid=1860564) 
(RayTrainWorker pid=1860564)   | Name    | Type             | Params | Mode 
(RayTrainWorker pid=1860564) -----------------------------------------------------
(RayTrainWorker pid=1860564) 0 | model   | Linear           | 125 K  | train
(RayTrainWorker pid=1860564) 1 | fc2     | Linear           | 150    | train
(RayTrainWorker pid=1860564) 2 | loss_fn | CrossEntropyLoss | 0      | train
(RayTrainWorker pid=1860564) -----------------------------------------------------
(RayTrainWorker pid=1860564) 125 K     Trainable params
(RayTrainWorker pid=1860564) 0         Non-trainable params
(RayTrainWorker pid=1860564) 125 K     Total params
(RayTrainWorker pid=1860564) 0.502     Total estimated model params size (MB)
(RayTrainWorker pid=1860564) 3         Modules in train mode
(RayTrainWorker pid=1860564) 0         Modules in eval mode
(RayTrainWorker pid=1860564) /mnt/hdd2/nam/miniconda3/envs

Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 10.61it/s]
                                                                           


(RayTrainWorker pid=1860564) /home/nam/protoplast/src/protoplast/scrna/anndata/torch_dataloader.py:115: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
(RayTrainWorker pid=1860564)   return torch.sparse_csr_tensor(
(RayTrainWorker pid=1860564) /mnt/hdd2/nam/miniconda3/envs/test/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:434: It is recommended to use `self.log('val_acc', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
(RayTrainWorker pid=1860564) /home/nam/protoplast/src/protoplast/scrna/anndata/torch_dataloader.py:115: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a featur

Epoch 0: 100%|██████████| 4224/4224 [01:25<00:00, 49.43it/s, v_num=0]
(RayTrainWorker pid=1860564) 
Validation: |          | 0/? [00:00<?, ?it/s]
(RayTrainWorker pid=1860564) 
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 1/960 [00:00<00:05, 159.84it/s]
(RayTrainWorker pid=1860564) 
Validation DataLoader 0:   1%|          | 5/960 [00:00<00:40, 23.62it/s]
(RayTrainWorker pid=1860564) 
Validation DataLoader 0:   1%|▏         | 12/960 [00:00<00:25, 37.67it/s]
(RayTrainWorker pid=1860564) 
Validation DataLoader 0:   1%|▏         | 13/960 [00:00<00:24, 39.29it/s]
(RayTrainWorker pid=1860564) 
Validation DataLoader 0:   2%|▏         | 22/960 [00:00<00:18, 49.67it/s]
(RayTrainWorker pid=1860564) 
Validation DataLoader 0:   3%|▎         | 30/960 [00:00<00:17, 53.87it/s]
(RayTrainWorker pid=1860564) 
Validation DataLoader 0:   3%|▎         | 31/960 [00:00<00:17, 54.48it/s]
(RayTrainWorker pid=1860564) 
Validation DataLoader 0:   3%|▎         | 32/960 [0

(RayTrainWorker pid=1860564) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/nam/protoplast_results/TorchTrainer_2025-09-26_10-15-09/TorchTrainer_ae708_00000_0_2025-09-26_10-15-09/checkpoint_000000)
(RayTrainWorker pid=1860564) `Trainer.fit` stopped: `max_epochs=1` reached.


In [35]:
result = trainer.train(DS_PATHS[0:1],
                       batch_size = 1024,
                       test_size = test_size, 
                       val_size = val_size,
                       num_workers = 1,
                       thread_per_worker = thread_per_worker)

Using 1 workers with {'CPU': 12} each


2025-09-26 10:15:09,441	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


=========Length of val_split 66 length of test_split 0 length of train_split 268
=========Warning: 0.09090909090909091 of data is dropped
=========Length of after dropping remainder val_split 60 length of test_split 0 length of train_split 264
Data splitting time: 10.16 seconds
Spawning Ray worker and initiating distributed training
== Status ==
Current time: 2025-09-26 10:15:09 (running for 00:00:00.12)
Using FIFO scheduling algorithm.
Logical resource usage: 0/96 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-09-26_10-14-51_243787_1828760/artifacts/2025-09-26_10-15-09/TorchTrainer_2025-09-26_10-15-09/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-09-26 10:15:14 (running for 00:00:05.17)
Using FIFO scheduling algorithm.
Logical resource usage: 13.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-09-26_10-14-51_243787_1828760/artifacts/2025-09-26_10-15-09/TorchTrainer_2025-09-2

2025-09-26 10:17:40,773	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/nam/protoplast_results/TorchTrainer_2025-09-26_10-15-09' in 0.0064s.
2025-09-26 10:17:40,778	INFO tune.py:1041 -- Total run time: 151.34 seconds (151.31 seconds for the tuning loop).


== Status ==
Current time: 2025-09-26 10:17:40 (running for 00:02:31.16)
Using FIFO scheduling algorithm.
Logical resource usage: 13.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-09-26_10-14-51_243787_1828760/artifacts/2025-09-26_10-15-09/TorchTrainer_2025-09-26_10-15-09/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2025-09-26 10:17:40 (running for 00:02:31.32)
Using FIFO scheduling algorithm.
Logical resource usage: 13.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-09-26_10-14-51_243787_1828760/artifacts/2025-09-26_10-15-09/TorchTrainer_2025-09-26_10-15-09/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)




In [36]:
ray.shutdown()